# Predicción futura

In [ ]:
import pandas as pd
import joblib

model = joblib.load("../models/model.pkl")

# Ejemplo hipotético:
future = pd.DataFrame({
    "temp": [28],
    "humidity": [60],
    "wind": [12],
    "hour": [14]
})

prediction = model.predict(future)
prediction
